In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json


In [2]:
# File to Load (Remember to Change These)
fileordprod = "./data/brazilian-ecommerce/olist_order_items_dataset.csv"

# Read Purchasing File and store into Pandas data frame
ordprod = pd.read_csv(fileordprod)[['order_id', 'product_id', 'price']]
print(ordprod.shape)
ordprod.head()

(112650, 3)


,order_id,product_id,price
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90


In [3]:
# File to Load (Remember to Change These)
fileordcusttime = "./data/brazilian-ecommerce/olist_orders_dataset.csv"

# Read Purchasing File and store into Pandas data frame
ordcusttime= pd.read_csv(fileordcusttime)[['order_id', 'customer_id', 'order_purchase_timestamp']]
print(ordcusttime.shape)
ordcusttime.head()

(99441, 3)


,order_id,customer_id,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39


In [4]:
# File to Load (Remember to Change These)
fileprodcat = "./data/brazilian-ecommerce/olist_products_dataset.csv"

# Read Purchasing File and store into Pandas data frame
prodcat = pd.read_csv(fileprodcat)[['product_id', 'product_category_name']]
prodcat.head()


,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [5]:
# File to Load (Remember to Change These)
fileprodtransl = "./data/brazilian-ecommerce/product_category_name_translation.csv"

# Read Purchasing File and store into Pandas data frame
prodtransl = pd.read_csv(fileprodtransl)
prodtransl.head()


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [6]:
import time
mergedf = pd.merge(ordcusttime, ordprod, on = 'order_id',how= 'left')
mergedf = pd.merge(mergedf, prodcat, on = 'product_id', how = 'left')
mergedf = pd.merge(mergedf, prodtransl, on = 'product_category_name', how = 'left')
renamecolumns = {
    'order_purchase_timestamp': 'order_time',
    'product_category_name_english': 'category'
}
mergedf = mergedf.rename(columns = renamecolumns)

mergedf ["date_time"]= pd.to_datetime(mergedf['order_time'])

#mergedf = mergedf[['order_purchase_timestamp', 'product_category_name_english', 'price']]

#renamecolumns = {
  #  'order_purchase_timestamp': 'time',
    #'product_category_name_english': 'category'
#}
#mergedf = mergedf.rename(columns = renamecolumns)
#print(mergedf.shape)
#len(mergedf['category'].unique())



In [7]:
mergedf['YYYY'] = pd.DatetimeIndex(mergedf['date_time']).year
mergedf['MM'] = pd.DatetimeIndex(mergedf['date_time']).month
mergedf

#converted to MM/YY Format
mergedf['month'] = pd.to_datetime(mergedf['date_time'])
mergedf['month'] = mergedf['month'].dt.strftime('%m/%y')

mergedf.head()

,order_id,customer_id,order_time,product_id,price,product_category_name,category,date_time,YYYY,MM,month
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,595fac2a385ac33a80bd5114aec74eb8,118.70,perfumaria,perfumery,2018-07-24 20:41:37,2018,7,07/18
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,aa4383b373c6aca5d8797843e5594415,159.90,automotivo,auto,2018-08-08 08:38:49,2018,8,08/18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,d0b61bfb1de832b15ba9d266ca96e5b0,45.00,pet_shop,pet_shop,2017-11-18 19:28:06,2017,11,11/17
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,65266b2da20d04dbe00c5c2d3bb7859e,19.90,papelaria,stationery,2018-02-13 21:18:39,2018,2,02/18


In [8]:
# URL for GET requests to retrieve data
urlCPI = "https://www.quandl.com/api/v3/datasets/BCB/191.json?api_key=2vRuu5GNchksqBdcAHzc"
urlBOV = "https://www.quandl.com/api/v3/datasets/BCB/7.json?api_key=2vRuu5GNchksqBdcAHzc"
urlEMP = "https://www.quandl.com/api/v3/datasets/BCB/10806.json?api_key=2vRuu5GNchksqBdcAHzc"

responseCPI = requests.get(urlCPI).json()
#print(json.dumps(response, indent=4, sort_keys=True))

responseBOV = requests.get(urlBOV).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [9]:
CPI_df = pd.DataFrame(responseCPI['dataset']['data'])

CPI_df = CPI_df.rename(index=str, columns={0: "month", 1: "CPI"})

CPI_df['month'] = pd.to_datetime(CPI_df['month'])
CPI_df['month'] = CPI_df['month'].dt.strftime('%m/%y')
#CPI_df.head()

,month,CPI
0,11/18,-0.17
1,10/18,0.48
2,09/18,0.45
3,08/18,0.07
4,07/18,0.17


In [11]:
BOV_df = pd.DataFrame(responseBOV['dataset']['data'])

BOV_df = BOV_df.rename(index=str, columns={0: "month", 1: "EMP"})

BOV_df['month'] = pd.to_datetime(BOV_df['month'])
BOV_df['month'] = BOV_df['month'].dt.strftime('%m/%y')
#BOV_df.head()

,month,EMP
0,12/18,87837.0
1,12/18,86977.0
2,12/18,86419.0
3,12/18,85914.0
4,12/18,88115.0


In [12]:
mergedf = pd.merge(mergedf, CPI_df, on = 'month', how = 'left')
mergedf = pd.merge(mergedf, BOV_df, on = 'month', how = 'left')

mergedf

(113425, 11)
(113425, 12)
(113425, 12)
(2343982, 13)


,order_id,customer_id,order_time,product_id,price,product_category_name,category,date_time,YYYY,MM,month,CPI,EMP
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,74308.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,74800.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,75975.0
3,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,75896.0
4,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,76671.0
5,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,76350.0
6,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,75413.0
7,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,76390.0
8,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,76283.0
9,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,87285b34884572647811a353c7ac498a,29.99,utilidades_domesticas,housewares,2017-10-02 10:56:33,2017,10,10/17,0.33,76591.0
